<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [187]:
import numpy as np
import pandas as pd

In [188]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [189]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [190]:
documentos = [documento.split() for documento in corpus]

for documento in documentos:
    print(documento)

['que', 'dia', 'es', 'hoy']
['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes']
['martes', 'muchas', 'gracias']


In [191]:
todos_los_documentos = [palabra for documento in documentos for palabra in documento]
vocabulario = sorted(set(todos_los_documentos))
vocabulario

['de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas', 'que']

### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [192]:
matriz_one_hot = np.zeros((len(corpus), len(vocabulario)))
matriz_one_hot

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [193]:
indice_palabra = {palabra: i for i, palabra in enumerate(vocabulario)}
indice_palabra

{'de': 0,
 'dia': 1,
 'el': 2,
 'es': 3,
 'gracias': 4,
 'hoy': 5,
 'martes': 6,
 'muchas': 7,
 'que': 8}

In [194]:
for idx, texto in enumerate(corpus):
    palabras = texto.split()
    for palabra in palabras:
        if palabra in vocabulario:
            col = indice_palabra[palabra]
            matriz_one_hot[idx, col] = 1

matriz_one_hot

array([[0., 1., 0., 1., 0., 1., 0., 0., 1.],
       [1., 1., 1., 1., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1., 0.]])

### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [205]:
frecuencias = {palabra: [texto.split().count(palabra) for texto in corpus] for palabra in vocabulario}
frecuencias

{'de': [0, 1, 0],
 'dia': [1, 1, 0],
 'el': [0, 1, 0],
 'es': [1, 1, 0],
 'gracias': [0, 0, 1],
 'hoy': [1, 1, 0],
 'martes': [0, 2, 1],
 'muchas': [0, 0, 1],
 'que': [1, 0, 0]}

In [196]:
df_tf_matrix = pd.DataFrame(frecuencias, index=corpus)
df_tf_matrix

,de,dia,el,es,gracias,hoy,martes,muchas,que
que dia es hoy,0,1,0,1,0,1,0,0,1
martes el dia de hoy es martes,1,1,1,1,0,1,2,0,0
martes muchas gracias,0,0,0,0,1,0,1,1,0


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [197]:
import math

In [198]:
tf_matrix = df_tf_matrix.values
tf_matrix

array([[0, 1, 0, 1, 0, 1, 0, 0, 1],
       [1, 1, 1, 1, 0, 1, 2, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0]])

In [199]:
matriz_one_hot

array([[0., 1., 0., 1., 0., 1., 0., 0., 1.],
       [1., 1., 1., 1., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1., 0.]])

In [200]:
idf_vector = np.zeros(len(vocabulario))
for idx, palabra in enumerate(vocabulario):
    documentos_con_palabra = sum(matriz_one_hot[:, idx])
    idf_vector[idx] = math.log(len(corpus)/documentos_con_palabra, 10)
idf_vector

array([0.47712125, 0.17609126, 0.47712125, 0.17609126, 0.47712125,
       0.17609126, 0.17609126, 0.47712125, 0.47712125])

In [201]:
tfidf_matrix = tf_matrix * idf_vector
df_tfidf = pd.DataFrame(tfidf_matrix, columns=vocabulario, index=corpus)
df_tfidf

,de,dia,el,es,gracias,hoy,martes,muchas,que
que dia es hoy,0.000000,0.176091,0.000000,0.176091,0.000000,0.176091,0.000000,0.000000,0.477121
martes el dia de hoy es martes,0.477121,0.176091,0.477121,0.176091,0.000000,0.176091,0.352183,0.000000,0.000000
martes muchas gracias,0.000000,0.000000,0.000000,0.000000,0.477121,0.000000,0.176091,0.477121,0.000000


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [202]:
def ordenar_documentos_por_similitud(corpus, indice_documento):
    todos_los_documentos = [palabra for documento in corpus for palabra in documento.split()]
    vocabulario = sorted(set(todos_los_documentos))
    
    matriz_one_hot = np.zeros((len(corpus), len(vocabulario)))
    indice_palabra = {palabra: i for i, palabra in enumerate(vocabulario)}
    
    for idx, texto in enumerate(corpus):
        palabras = texto.split()
        for palabra in palabras:
            if palabra in vocabulario:
                col = indice_palabra[palabra]
                matriz_one_hot[idx, col] = 1
    
    documento_seleccionado = matriz_one_hot[indice_documento, :]
    similitudes = [cosine_similarity(documento_seleccionado, matriz_one_hot[i, :]) for i in range(len(corpus))]
    
    documentos_ordenados = [x for _, x in sorted(zip(similitudes, corpus), reverse=True)]
    
    return documentos_ordenados

In [203]:
documentos_ordenados = ordenar_documentos_por_similitud(corpus, 1)
documentos_ordenados

['martes el dia de hoy es martes', 'que dia es hoy', 'martes muchas gracias']